# Customer Segmentation - Data Prep

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from tabulate import tabulate
import warnings
from functools import reduce
from sklearn.impute import KNNImputer
from sklearn.manifold import TSNE
from sklearn.preprocessing import MinMaxScaler
import sklearn.cluster as cluster
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score, adjusted_mutual_info_score
from sklearn.metrics import silhouette_score
from sklearn.decomposition import PCA

import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import animation

from functools import reduce

%matplotlib inline 

warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None) 
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', 200)

base_path = 'data/'

### helpers

In [2]:
def na_check(df, na_only = True): 
    na = df.isna().mean()*100
    if na_only == False:
        print(na)
    else:
        print(na[na>0])

## Ricoh Opps

In [3]:
base_path = 'data/'
opp = pd.read_csv(f'{base_path}oppo.csv')
lead = pd.read_csv(f'{base_path}lead.csv')

In [4]:
opp.columns

Index(['key', 'opportunity_number', 'quote_number', 'order_number',
       'l2_rise_number', 'funnel_stage', 'customer_segment', 'crm_industry',
       'a4_bw_flag', 'a4_c_flag', 'a3_bw_flag', 'a3_c_flag', 'docuware_flag',
       'ransomcare_flag', 'it_health_check_flag', 'ms_flag', 'pp_bw_cs_flag',
       'pp_c_cs_flag', 'wf_bw_flag', 'wf_c_flag', 'bw_lp_flag',
       'color_lp_flag', 'mfp_flag', 'mindshift_flag', 'opp_date', 'opp_rev',
       'quote_date', 'quote_rev', 'order_date', 'order_rev', 'close_date',
       'sales_stage_status', 'start_date', 'open_days', 'close_days', 'win',
       'rev', 'eloqua', 'eloqua oc', 'lead', 'opp', 'order', 'quote', 'sales',
       'web', 'webinar', 'cs', 'hw', 'its', 'ms off', 'ms on', 'ps', 'refi',
       'rent', 'supp', 'ts', 'svc_calls', 'cv', 'mif', 'renewal', 'sat', 'nps',
       'department', 'job_type', 'employees_dnb', 'sales_dnb'],
      dtype='object')

### get column categories

In [5]:
# categorize
print(opp.columns.get_loc("eloqua"))
print(opp.columns.get_loc("webinar"))
print(opp.columns.get_loc("sat"))
print(opp.columns.get_loc("cs"))
print(opp.columns.get_loc("renewal"))
eng_cols = opp.columns[37:46].tolist()
sat_cols = opp.columns[60:62].tolist()
key_cols = ['key', 'opportunity_number', 'lead_number', 'quote_number', 'order_number', 'l2_rise_number']
cat_cols = opp.select_dtypes(include=['object']).columns
inv_cols = opp.columns[53:67].tolist()
flag_cols = [col for col in opp.columns if "flag" in col]
rev_cols = [col for col in opp.columns if 'rev' in col]

37
45
60
46
59


In [6]:
na_check(opp)

quote_number    29.477635
order_number    53.153378
quote_date      29.477635
order_date      77.588368
sat             84.738623
nps             84.738623
dtype: float64


In [7]:
# drop columns that all values are the same or other columns consist information to cover
opp.loc[:, opp.nunique() == 1].columns
opp.drop(columns=opp.columns[opp.nunique() == 1], inplace=True)
opp.drop(columns = ['quote_number', 'order_number', 'quote_date', 'order_date'], inplace = True)

## MVP Stage of clustering: by company
Each compony could have different lead/opp happended in the past, each intention to buy can be viewed as a lead/opp.
So first, group variables into types: 
1. consistent information regarding the company: l2_rise_number, customer_segment, crm_industry, employee_dnb, sales_dnb
2. progression stats: from eloqua to renewal, .. webinar, sat, nps
3. specific to this opp: remaining vars: numbers, dates, funnel_stage, sales_stage_status, lead related, flags, revs, cs - renewal
4. feature engineer: tenure, 

### aggregate values for each company

In [8]:
con_cols = ['l2_rise_number', 'customer_segment', 'crm_industry', 'employees_dnb', 'sales_dnb']
pro_cols = eng_cols + sat_cols
key_cols = ['key', 'opportunity_number', 'lead_number', 'quote_number', 'order_number']
date_cols = [col for col in opp.columns if 'date' in col]
opp_cols = list(set(opp.columns.tolist()) - set(con_cols) - set(pro_cols) - set(key_cols) - set(date_cols))
opp_num_cols = opp[opp_cols].select_dtypes(include=['number']).columns
print(opp_num_cols)
opp_cat_cols = opp[opp_cols].select_dtypes(include=['object']).columns
print(opp_cat_cols)

Index(['pp_c_cs_flag', 'color_lp_flag', 'cs', 'a3_bw_flag', 'pp_bw_cs_flag',
       'its', 'opp_rev', 'open_days', 'renewal', 'a4_bw_flag', 'rent', 'hw',
       'supp', 'ts', 'mindshift_flag', 'ms on', 'refi', 'svc_calls',
       'wf_c_flag', 'a3_c_flag', 'ms_flag', 'a4_c_flag', 'wf_bw_flag',
       'close_days', 'it_health_check_flag', 'rev', 'mfp_flag', 'bw_lp_flag',
       'docuware_flag', 'mif', 'ms off', 'order_rev', 'quote_rev', 'cv', 'ps',
       'win', 'ransomcare_flag'],
      dtype='object')
Index(['department', 'sales_stage_status', 'job_type'], dtype='object')


In [9]:
#1. choose 1 for consistent cols
l2_con = opp.sort_values(by = 'start_date')[con_cols].groupby('l2_rise_number').last().reset_index()

#2. get max for engagement levels (an approximation of one-year engagement) 
l2_pro = opp.groupby('l2_rise_number')[eng_cols].agg('max').reset_index()

# 3. features for deal-related: ideally want it be weighted, but need to define how to agg
l2_opp_num = opp.groupby('l2_rise_number')[opp_num_cols].mean().reset_index()

# 4. normalized frequency for all categorical variables
comp = "l2_rise_number"
res = []
grouped = opp.groupby(comp)

for col in opp_cat_cols:
    col_res = grouped[col].apply(lambda x: x.value_counts(normalize=True) if not x.isna().all() else pd.Series([0], index=[np.nan]))
    col_res = col_res.unstack(fill_value=0).add_prefix(f'{col}_')
    res.append(col_res)

l2_opp_cat = pd.concat(res, axis=1)
l2_opp_cat.reset_index(inplace=True)

In [10]:
# 5. feature engineer:
opp['start_date'] = pd.to_datetime(opp['start_date'], format='%Y-%m-%d')
# tenure 
cur_ts = opp.start_date.max()
tenure = opp.groupby('l2_rise_number')['start_date'].min().reset_index()
tenure['tenure_yr'] = (cur_ts - tenure['start_date']).dt.days/365.25

# email click rate: 
l2_pro['eloqua_rt'] = np.where(l2_pro['eloqua'] != 0, l2_pro['eloqua oc'] / l2_pro['eloqua'], 0)

In [11]:
# 6. join them to one df

dfs = [l2_con, l2_pro, l2_opp_num, l2_opp_cat, tenure]  
l2 = reduce(lambda left, right: pd.merge(left, right, on = "l2_rise_number", how = "inner"), dfs)

na_check(l2, True)

Series([], dtype: float64)


### add label for deals

In [12]:
# ranking for the largest deal of each comp 
top = opp.groupby('l2_rise_number').order_rev.max().reset_index()
print(100*top[top.order_rev ==0].shape[0]/top.shape[0], "% of companies have 0 order_rev")

76.89062688326123 % of companies have 0 order_rev


In [13]:
unique_values = top['order_rev'].unique()

# Get thresholds for largest 10, top 1%, and top 10% values
n_10 = top['order_rev'].nlargest(10).min()
top_01p = np.percentile(top['order_rev'], 99.9)
top_1p = np.percentile(top['order_rev'], 99)
top_10p = np.percentile(top['order_rev'], 90)

# Ensure bins are unique and monotonically increasing
bins = sorted(set([-1, 0, top_1p, top_01p, top_10p, n_10, np.inf]))

# Define labels, one less than bins
labels = ['0', 'rest', 'top_10p', 'top_1p', 'top_01p', 'n_10']

top['max_deal'] = pd.cut(top['order_rev'], bins=bins, labels=labels, right=False).fillna('rest')
top['max_deal'] = np.where(top['order_rev'] == 0, '0', top['max_deal'])

In [14]:
#top['l2_rise_number'].nunique()
l2.l2_rise_number.nunique()

74671

In [15]:
top.groupby('max_deal').size()

max_deal
0          57415
n_10          10
rest        9788
top_01p       65
top_10p     6721
top_1p       672
dtype: int64

In [16]:
l2 = pd.merge(l2, top[['l2_rise_number', 'max_deal']], on = 'l2_rise_number', how = 'inner')
l2.shape

(74671, 87)

In [17]:
l2['max_deal'] = l2['max_deal'].astype(str)

In [18]:
# check point -- save file for later
l2.to_csv(f'{base_path}l2_cleaned.csv', index = False)

In [ ]:
se = manifold.SpectralEmbedding(
    n_components=2, n_neighbors=n_neighbors, random_state=42
)
trans_data = se.fit_transform(X).T
t1 = time()
print("Spectral Embedding: %.2g sec" % (t1 - t0))

ax = fig.add_subplot(259)
plt.scatter(X[0], X[1], c=colors, cmap=plt.cm.rainbow)
plt.title("Spectral Embedding (%.2g sec)" % (t1 - t0))
ax.xaxis.set_major_formatter(NullFormatter())
ax.yaxis.set_major_formatter(NullFormatter())
plt.axis("tight")